In [1]:
import ROOT
from root_utils_v2 import AGC_Sample, RebinningTool, MuteTool, Visualization

/shared/vkholoim/agc_root_inter/cms_ttbar_analysis/root_utils_v2.py:4: RuntimeWarning: class "RooStats::HistFactory::Sample" has no virtual destructor
  class AGC_Sample(ROOT.RooStats.HistFactory.Sample):


In [2]:
MuteTool() # Mute all the messages from minimizer

In [3]:
rebinning_tool = RebinningTool() # Rebinning as extra step (was discussed with Jonas)
rebinning_tool.set_xmin(110)
rebinning_tool.set_rebin(2)
rebinning_tool.set_input_path("data/histograms.root")
rebinning_tool.set_output_path("data/temp_histos.root") # if you dont specify this -> overwrite (?)
rebinning_tool.apply_rebinning()

In [4]:
file = ROOT.TFile("ROOT_AGC_Utils/HistFactoryExtra.root", "RECREATE") # should be hidden. Cleaning the file
file.Close() # create separate file for each file -> two much files

input_file = "data/temp_histos.root" # input file for futher defined histograms

In [5]:
meas = ROOT.RooStats.HistFactory.Measurement("meas", "meas")
meas.SetLumi(1.0)
meas.SetLumiRelErr(0.0)

# ! main difference
lumi_systematics = ROOT.RooStats.HistFactory.OverallSys() # we define it one time -> dont need to redefine it
lumi_systematics.SetName("Lumi")
lumi_systematics.SetLow(0.97)
lumi_systematics.SetHigh(1.03)

In [6]:
channel = ROOT.RooStats.HistFactory.Channel("channel_4j1b_CR")
channel.SetData("4j1b_pseudodata", input_file)
channel.SetStatErrorConfig(0.001, "Gaussian")

ttbar = AGC_Sample("ttbar", "4j1b_ttbar", input_file)
ttbar.SetSystematicsInputFile(input_file)   # if use SetInputFile -> change input file for nominal histogram 
                                            # we can put input file from sample data definition as default

ttbar.AddOverallSys(lumi_systematics)
ttbar.ActivateStatError() # ROOT does not have DisableStatError() method, so we have to keep this :(
ttbar.AddNormPlusShapeHistoSys("ME_variation",      histoname_up = "4j1b_ttbar_ME_var"                                                          ) # "histogram_up" can be skipped here
ttbar.AddNormPlusShapeHistoSys("PS_variation",      histoname_up = "4j1b_ttbar_PS_var"                                                          ) # histogram path is specified as "" in default case
ttbar.AddNormPlusShapeHistoSys("tt_scale_var",      histoname_up = "4j1b_ttbar_scaleup",        histoname_down = "4j1b_ttbar_scaledown"         )
ttbar.AddNormPlusShapeHistoSys("jet_energy_scale",  histoname_up = "4j1b_ttbar_pt_scale_up"                                                     )
ttbar.AddNormPlusShapeHistoSys("jet_energy_res",    histoname_up = "4j1b_ttbar_pt_res_up"                                                       )
ttbar.AddNormPlusShapeHistoSys("b_tag_NP_1",        histoname_up = "4j1b_ttbar_btag_var_0_up",  histoname_down = "4j1b_ttbar_btag_var_0_down"   )
ttbar.AddNormPlusShapeHistoSys("b_tag_NP_2",        histoname_up = "4j1b_ttbar_btag_var_1_up",  histoname_down = "4j1b_ttbar_btag_var_1_down"   )
ttbar.AddNormPlusShapeHistoSys("b_tag_NP_3",        histoname_up = "4j1b_ttbar_btag_var_2_up",  histoname_down = "4j1b_ttbar_btag_var_2_down"   )
ttbar.AddNormPlusShapeHistoSys("b_tag_NP_4",        histoname_up = "4j1b_ttbar_btag_var_3_up",  histoname_down = "4j1b_ttbar_btag_var_3_down"   )
ttbar.AddNormFactor("ttbar_norm", 1, 0, 10)
channel.AddSample(ttbar)

wjets = AGC_Sample("wjets", "4j1b_wjets", input_file)
wjets.SetSystematicsInputFile(input_file)
wjets.ActivateStatError()
wjets.AddOverallSys(lumi_systematics)
wjets.AddNormPlusShapeHistoSys("jet_energy_scale",      histoname_up = "4j1b_wjets_pt_scale_up"                                                     )
wjets.AddNormPlusShapeHistoSys("jet_energy_res",        histoname_up = "4j1b_wjets_pt_res_up"                                                       )
wjets.AddNormPlusShapeHistoSys("b_tag_NP_1",            histoname_up = "4j1b_wjets_btag_var_0_up",  histoname_down = "4j1b_wjets_btag_var_0_down"   )
wjets.AddNormPlusShapeHistoSys("b_tag_NP_2",            histoname_up = "4j1b_wjets_btag_var_1_up",  histoname_down = "4j1b_wjets_btag_var_1_down"   )
wjets.AddNormPlusShapeHistoSys("b_tag_NP_3",            histoname_up = "4j1b_wjets_btag_var_2_up",  histoname_down = "4j1b_wjets_btag_var_2_down"   )
wjets.AddNormPlusShapeHistoSys("b_tag_NP_4",            histoname_up = "4j1b_wjets_btag_var_3_up",  histoname_down = "4j1b_wjets_btag_var_3_down"   )
wjets.AddNormPlusShapeHistoSys("w_plus_jets_scale_var", histoname_up = "4j1b_wjets_scale_var_up",   histoname_down = "4j1b_wjets_scale_var_down"    )
channel.AddSample(wjets)

single_top_s_chan = AGC_Sample("single_top_s", "4j1b_single_top_s_chan", input_file)
single_top_s_chan.SetSystematicsInputFile(input_file)
single_top_s_chan.ActivateStatError()
single_top_s_chan.AddOverallSys(lumi_systematics)
single_top_s_chan.AddNormPlusShapeHistoSys("jet_energy_scale",  histoname_up = "4j1b_single_top_s_chan_pt_scale_up"                                                                 )
single_top_s_chan.AddNormPlusShapeHistoSys("jet_energy_res",    histoname_up = "4j1b_single_top_s_chan_pt_res_up"                                                                   )
single_top_s_chan.AddNormPlusShapeHistoSys("b_tag_NP_1",        histoname_up = "4j1b_single_top_s_chan_btag_var_0_up",  histoname_down = "4j1b_single_top_s_chan_btag_var_0_down"   )
single_top_s_chan.AddNormPlusShapeHistoSys("b_tag_NP_2",        histoname_up = "4j1b_single_top_s_chan_btag_var_1_up",  histoname_down = "4j1b_single_top_s_chan_btag_var_1_down"   )
single_top_s_chan.AddNormPlusShapeHistoSys("b_tag_NP_3",        histoname_up = "4j1b_single_top_s_chan_btag_var_2_up",  histoname_down = "4j1b_single_top_s_chan_btag_var_2_down"   )
single_top_s_chan.AddNormPlusShapeHistoSys("b_tag_NP_4",        histoname_up = "4j1b_single_top_s_chan_btag_var_3_up",  histoname_down = "4j1b_single_top_s_chan_btag_var_3_down"   )

channel.AddSample(single_top_s_chan)

single_top_t_chan = AGC_Sample("single_top_t", "4j1b_single_top_t_chan", input_file)
single_top_t_chan.SetSystematicsInputFile(input_file)
single_top_t_chan.ActivateStatError()
single_top_t_chan.AddOverallSys(lumi_systematics)
single_top_t_chan.AddNormPlusShapeHistoSys("jet_energy_scale",  histoname_up = "4j1b_single_top_t_chan_pt_scale_up"                                                              )
single_top_t_chan.AddNormPlusShapeHistoSys("jet_energy_res",    histoname_up = "4j1b_single_top_t_chan_pt_res_up"                                                                )
single_top_t_chan.AddNormPlusShapeHistoSys("b_tag_NP_1",        histoname_up = "4j1b_single_top_t_chan_btag_var_0_up",  histoname_down = "4j1b_single_top_t_chan_btag_var_0_down")
single_top_t_chan.AddNormPlusShapeHistoSys("b_tag_NP_2",        histoname_up = "4j1b_single_top_t_chan_btag_var_1_up",  histoname_down = "4j1b_single_top_t_chan_btag_var_1_down")
single_top_t_chan.AddNormPlusShapeHistoSys("b_tag_NP_3",        histoname_up = "4j1b_single_top_t_chan_btag_var_2_up",  histoname_down = "4j1b_single_top_t_chan_btag_var_2_down")
single_top_t_chan.AddNormPlusShapeHistoSys("b_tag_NP_4",        histoname_up = "4j1b_single_top_t_chan_btag_var_3_up",  histoname_down = "4j1b_single_top_t_chan_btag_var_3_down")

channel.AddSample(single_top_t_chan)

single_top_tW = AGC_Sample("single_top_tW", "4j1b_single_top_tW", input_file)
single_top_tW.SetSystematicsInputFile(input_file)
single_top_tW.ActivateStatError()
single_top_tW.AddOverallSys(lumi_systematics)
single_top_tW.AddNormPlusShapeHistoSys("jet_energy_scale",  histoname_up = "4j1b_single_top_tW_pt_scale_up"                                                                 )
single_top_tW.AddNormPlusShapeHistoSys("jet_energy_res",    histoname_up =  "4j1b_single_top_tW_pt_res_up"                                                                  )               
single_top_tW.AddNormPlusShapeHistoSys("b_tag_NP_1",        histoname_up = "4j1b_single_top_tW_btag_var_0_up",      histoname_down = "4j1b_single_top_tW_btag_var_0_down"   )
single_top_tW.AddNormPlusShapeHistoSys("b_tag_NP_2",        histoname_up = "4j1b_single_top_tW_btag_var_1_up",      histoname_down = "4j1b_single_top_tW_btag_var_1_down"   )
single_top_tW.AddNormPlusShapeHistoSys("b_tag_NP_3",        histoname_up = "4j1b_single_top_tW_btag_var_2_up",      histoname_down = "4j1b_single_top_tW_btag_var_2_down"   )
single_top_tW.AddNormPlusShapeHistoSys("b_tag_NP_4",        histoname_up = "4j1b_single_top_tW_btag_var_3_up",      histoname_down = "4j1b_single_top_tW_btag_var_3_down"   )

channel.AddSample(single_top_tW)

meas.AddChannel(channel)

channel_2b = ROOT.RooStats.HistFactory.Channel("channel_4j2b_SR")
channel_2b.SetData("4j2b_pseudodata", input_file)
channel_2b.SetStatErrorConfig(0.001, "Gaussian")

ttbar = AGC_Sample("ttbar", "4j2b_ttbar", input_file)
ttbar.AddOverallSys(lumi_systematics)
ttbar.SetSystematicsInputFile(input_file)
ttbar.ActivateStatError()
ttbar.AddNormPlusShapeHistoSys("ME_variation",      histoname_up = "4j2b_ttbar_ME_var"                                                          )
ttbar.AddNormPlusShapeHistoSys("PS_variation",      histoname_up = "4j2b_ttbar_PS_var"                                                          )
ttbar.AddNormPlusShapeHistoSys("tt_scale_var",      histoname_up = "4j2b_ttbar_scaleup",        histoname_down = "4j2b_ttbar_scaledown"         )
ttbar.AddNormPlusShapeHistoSys("jet_energy_scale",  histoname_up = "4j2b_ttbar_pt_scale_up"                                                     )
ttbar.AddNormPlusShapeHistoSys("jet_energy_res",    histoname_up = "4j2b_ttbar_pt_res_up"                                                       )
ttbar.AddNormPlusShapeHistoSys("b_tag_NP_1",        histoname_up = "4j2b_ttbar_btag_var_0_up",  histoname_down = "4j2b_ttbar_btag_var_0_down"   )
ttbar.AddNormPlusShapeHistoSys("b_tag_NP_2",        histoname_up = "4j2b_ttbar_btag_var_1_up",  histoname_down = "4j2b_ttbar_btag_var_1_down"   )
ttbar.AddNormPlusShapeHistoSys("b_tag_NP_3",        histoname_up = "4j2b_ttbar_btag_var_2_up",  histoname_down = "4j2b_ttbar_btag_var_2_down"   )
ttbar.AddNormPlusShapeHistoSys("b_tag_NP_4",        histoname_up = "4j2b_ttbar_btag_var_3_up",  histoname_down = "4j2b_ttbar_btag_var_3_down"   )
ttbar.AddNormFactor("ttbar_norm", 1, 0, 10)
channel_2b.AddSample(ttbar)

wjets = AGC_Sample("wjets", "4j2b_wjets", input_file)
wjets.SetSystematicsInputFile(input_file)
wjets.ActivateStatError()
wjets.AddOverallSys(lumi_systematics)
wjets.AddNormPlusShapeHistoSys("jet_energy_scale",      histoname_up = "4j2b_wjets_pt_scale_up"                                                     )
wjets.AddNormPlusShapeHistoSys("jet_energy_res",        histoname_up = "4j2b_wjets_pt_res_up"                                                       )
wjets.AddNormPlusShapeHistoSys("b_tag_NP_1",            histoname_up = "4j2b_wjets_btag_var_0_up",  histoname_down = "4j2b_wjets_btag_var_0_down"   )
wjets.AddNormPlusShapeHistoSys("b_tag_NP_2",            histoname_up = "4j2b_wjets_btag_var_1_up",  histoname_down = "4j2b_wjets_btag_var_1_down"   )
wjets.AddNormPlusShapeHistoSys("b_tag_NP_3",            histoname_up = "4j2b_wjets_btag_var_2_up",  histoname_down = "4j2b_wjets_btag_var_2_down"   )
wjets.AddNormPlusShapeHistoSys("b_tag_NP_4",            histoname_up = "4j2b_wjets_btag_var_3_up",  histoname_down = "4j2b_wjets_btag_var_3_down"   )
wjets.AddNormPlusShapeHistoSys("w_plus_jets_scale_var", histoname_up = "4j2b_wjets_scale_var_up",   histoname_down = "4j2b_wjets_scale_var_down"    )
channel_2b.AddSample(wjets)

single_top_s_chan = AGC_Sample("single_top_s", "4j2b_single_top_s_chan", input_file)
single_top_s_chan.SetSystematicsInputFile(input_file)
single_top_s_chan.ActivateStatError()
single_top_s_chan.AddOverallSys(lumi_systematics)
single_top_s_chan.AddNormPlusShapeHistoSys("jet_energy_scale",  histoname_up = "4j2b_single_top_s_chan_pt_scale_up"                                                                 )
single_top_s_chan.AddNormPlusShapeHistoSys("jet_energy_res",    histoname_up = "4j2b_single_top_s_chan_pt_res_up"                                                                   )
single_top_s_chan.AddNormPlusShapeHistoSys("b_tag_NP_1",        histoname_up = "4j2b_single_top_s_chan_btag_var_0_up",  histoname_down = "4j2b_single_top_s_chan_btag_var_0_down"   )
single_top_s_chan.AddNormPlusShapeHistoSys("b_tag_NP_2",        histoname_up = "4j2b_single_top_s_chan_btag_var_1_up",  histoname_down = "4j2b_single_top_s_chan_btag_var_1_down"   )
single_top_s_chan.AddNormPlusShapeHistoSys("b_tag_NP_3",        histoname_up = "4j2b_single_top_s_chan_btag_var_2_up",  histoname_down = "4j2b_single_top_s_chan_btag_var_2_down"   )
single_top_s_chan.AddNormPlusShapeHistoSys("b_tag_NP_4",        histoname_up = "4j2b_single_top_s_chan_btag_var_3_up",  histoname_down = "4j2b_single_top_s_chan_btag_var_3_down"   )
channel_2b.AddSample(single_top_s_chan)

single_top_t_chan = AGC_Sample("single_top_t", "4j2b_single_top_t_chan", input_file)
single_top_t_chan.SetSystematicsInputFile(input_file)
single_top_t_chan.ActivateStatError()
single_top_t_chan.AddOverallSys(lumi_systematics)
single_top_t_chan.AddNormPlusShapeHistoSys("jet_energy_scale",  histoname_up = "4j2b_single_top_t_chan_pt_scale_up"                                                                 )
single_top_t_chan.AddNormPlusShapeHistoSys("jet_energy_res",    histoname_up = "4j2b_single_top_t_chan_pt_res_up"                                                                   )
single_top_t_chan.AddNormPlusShapeHistoSys("b_tag_NP_1",        histoname_up = "4j2b_single_top_t_chan_btag_var_0_up",  histoname_down = "4j2b_single_top_t_chan_btag_var_0_down"   )
single_top_t_chan.AddNormPlusShapeHistoSys("b_tag_NP_2",        histoname_up = "4j2b_single_top_t_chan_btag_var_1_up",  histoname_down = "4j2b_single_top_t_chan_btag_var_1_down"   )
single_top_t_chan.AddNormPlusShapeHistoSys("b_tag_NP_3",        histoname_up = "4j2b_single_top_t_chan_btag_var_2_up",  histoname_down = "4j2b_single_top_t_chan_btag_var_2_down"   )
single_top_t_chan.AddNormPlusShapeHistoSys("b_tag_NP_4",        histoname_up = "4j2b_single_top_t_chan_btag_var_3_up",  histoname_down = "4j2b_single_top_t_chan_btag_var_3_down"   )

channel_2b.AddSample(single_top_t_chan)

single_top_tW = AGC_Sample("single_top_tW", "4j2b_single_top_tW", input_file)
single_top_tW.ActivateStatError()
single_top_tW.SetSystematicsInputFile(input_file)
single_top_tW.AddOverallSys(lumi_systematics)
single_top_tW.AddNormPlusShapeHistoSys("jet_energy_scale",  histoname_up = "4j2b_single_top_tW_pt_scale_up"                                                             )
single_top_tW.AddNormPlusShapeHistoSys("jet_energy_res",    histoname_up = "4j2b_single_top_tW_pt_res_up"                                                               )
single_top_tW.AddNormPlusShapeHistoSys("b_tag_NP_1",        histoname_up = "4j2b_single_top_tW_btag_var_0_up",  histoname_down = "4j2b_single_top_tW_btag_var_0_down"   )
single_top_tW.AddNormPlusShapeHistoSys("b_tag_NP_2",        histoname_up = "4j2b_single_top_tW_btag_var_1_up",  histoname_down = "4j2b_single_top_tW_btag_var_1_down"   )
single_top_tW.AddNormPlusShapeHistoSys("b_tag_NP_3",        histoname_up = "4j2b_single_top_tW_btag_var_2_up",  histoname_down = "4j2b_single_top_tW_btag_var_2_down"   )
single_top_tW.AddNormPlusShapeHistoSys("b_tag_NP_4",        histoname_up = "4j2b_single_top_tW_btag_var_3_up",  histoname_down = "4j2b_single_top_tW_btag_var_3_down"   )

channel_2b.AddSample(single_top_tW)
meas.AddChannel(channel_2b)

AGC_Sample instance ttbar is being destroyed
AGC_Sample instance wjets is being destroyed
AGC_Sample instance single_top_s is being destroyed
AGC_Sample instance single_top_t is being destroyed
AGC_Sample instance single_top_tW is being destroyed


In [7]:
meas.SetPOI("ttbar_norm")
meas.CollectHistograms()
meas.SetExportOnly(True)

[#2] PROGRESS:HistFactory -- Getting histogram data/temp_histos.root:/4j1b_pseudodata
[#2] INFO:HistFactory -- Opened input file: data/temp_histos.root: 
[#2] PROGRESS:HistFactory -- Getting histogram data/temp_histos.root:/4j1b_ttbar
[#2] PROGRESS:HistFactory -- Getting histogram ROOT_AGC_Utils/HistFactoryExtra.root:/4j1b_ttbar_ME_variation_norm_plus_shape_down
[#2] INFO:HistFactory -- Opened input file: ROOT_AGC_Utils/HistFactoryExtra.root: 
[#2] PROGRESS:HistFactory -- Getting histogram ROOT_AGC_Utils/HistFactoryExtra.root:/4j1b_ttbar_ME_variation_norm_plus_shape_up
[#2] PROGRESS:HistFactory -- Getting histogram ROOT_AGC_Utils/HistFactoryExtra.root:/4j1b_ttbar_PS_variation_norm_plus_shape_down
[#2] PROGRESS:HistFactory -- Getting histogram ROOT_AGC_Utils/HistFactoryExtra.root:/4j1b_ttbar_PS_variation_norm_plus_shape_up
[#2] PROGRESS:HistFactory -- Getting histogram ROOT_AGC_Utils/HistFactoryExtra.root:/4j1b_ttbar_tt_scale_var_norm_plus_shape_down
[#2] PROGRESS:HistFactory -- Getting

In [8]:
ws = ROOT.RooStats.HistFactory.MakeModelAndMeasurementFast(meas)

# Retrieve the ModelConfig
modelConfig = ws.obj("ModelConfig")


# Extract the PDF and global observables
pdf = modelConfig.GetPdf()
globalObservables = ROOT.RooArgSet(modelConfig.GetGlobalObservables())


# Perform the fit
result = pdf.fitTo(ws.data("obsData"), ROOT.RooFit.Save(), ROOT.RooFit.PrintLevel(-1), ROOT.RooFit.GlobalObservables(globalObservables))

# Print the fit result
result.Print()

[#2] INFO:HistFactory -- Making Model and Measurements (Fast) for measurement: meas
[#2] INFO:HistFactory -- using lumi = 1 and lumiError = 0 including bins between 0 and 1
[#2] INFO:HistFactory -- fixing the following parameters:
[#2] INFO:HistFactory -- Creating the output file: _meas.root
[#2] INFO:HistFactory -- Creating the HistoToWorkspaceFactoryFast factory
[#2] INFO:HistFactory -- Setting preprocess functions
[#2] PROGRESS:HistFactory -- Starting to process channel: channel_4j1b_CR
[#2] PROGRESS:HistFactory -- 
-----------------------------------------
	Starting to process 'channel_4j1b_CR' channel with 1 observables
-----------------------------------------

[#2] INFO:HistFactory -- making normFactor: ttbar_norm
[#2] INFO:HistFactory -- processing hist 4j1b_ttbartemp_rebin_clone
[#2] INFO:HistFactory -- Sample: ttbar to be included in Stat Error for channel channel_4j1b_CR
[#2] INFO:HistFactory -- Making Statistical Uncertainty Hist for  Channel: channel_4j1b_CR Sample: ttbar


In [10]:
vis = Visualization()
vis.CreateAndSavePicture("picture.png", ws) # Parameters fit result

Name: combined Title: combined
{ @0x558044965690, @0x558044965698, @0x5580449656a0, @0x5580449656a8, @0x5580449656b0, @0x5580449656b8, @0x5580449656c0, @0x5580449656c8, @0x5580449656d0, @0x5580449656d8, @0x5580449656e0, @0x5580449656e8, @0x5580449656f0, @0x5580449656f8, @0x558044965700, @0x558044965708, @0x558044965710, @0x558044965718, @0x558044965720, @0x558044965728, @0x558044965730, @0x558044965738, @0x558044965740, @0x558044965748, @0x558044965750, @0x558044965758, @0x558044965760, @0x558044965768, @0x558044965770, @0x558044965778, @0x558044965780, @0x558044965788, @0x558044965790, @0x558044965798, @0x5580449657a0, @0x5580449657a8, @0x5580449657b0, @0x5580449657b8, @0x5580449657c0, @0x5580449657c8, @0x5580449657d0, @0x5580449657d8, @0x5580449657e0, @0x5580449657e8, @0x5580449657f0, @0x5580449657f8, @0x558044965800, @0x558044965808, @0x558044965810, @0x558044965818, @0x558044965820, @0x558044965828, @0x558044965830, @0x558044965838, @0x558044965840, @0x558044965848, @0x558044965850

Info in <TCanvas::Print>: png file picture.png has been created
